In [20]:
!pip install langchain langchain_community langchain_openai

  Attempting uninstall: openai
    Found existing installation: openai 1.51.0
    Uninstalling openai-1.51.0:
      Successfully uninstalled openai-1.51.0


# Setup OpenAI 

In [1]:
import getpass
import os

os.environ['OPENAI_API_KEY'] = getpass.getpass("OpenAI API Key: ")

OpenAI API Key:  ········


In [37]:
from pathlib import Path

from langchain.agents import Tool, AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import SimpleDirectoryReader,VectorStoreIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [4]:
transformers_path = Path("./data/research_papers/transformers")
cnn_path = Path("./data/research_papers/cnn")
rag_path = Path("./data/research_papers/rag")

In [5]:
# load documents from research papers
transformers_docs = SimpleDirectoryReader(transformers_path).load_data()
cnn_docs = SimpleDirectoryReader(cnn_path).load_data()
rag_docs = SimpleDirectoryReader(rag_path).load_data()

In [6]:
# build index from research papers documents
transformers_index = VectorStoreIndex.from_documents(transformers_docs)
cnn_index = VectorStoreIndex.from_documents(cnn_docs)
rag_index = VectorStoreIndex.from_documents(rag_docs)

In [7]:
#Creating Query engines from documents
transformers_engine = transformers_index.as_query_engine(similarity_top_k=3)
cnn_engine = cnn_index.as_query_engine(similarity_top_k=3)
rag_engine = rag_index.as_query_engine(similarity_top_k=3)

In [31]:
#creating tools for research paper query engines
query_engine_tools = [

    QueryEngineTool(
        query_engine=transformers_engine,
        metadata=ToolMetadata(
            name="transformers_papers",
            description=(
                "Provides information about Transformers and related concepts"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=cnn_engine,
        metadata=ToolMetadata(
            name="cnn_papers",
            description=(
                "Provides information about CNN and related architectures"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=cnn_engine,
        metadata=ToolMetadata(
            name="RAG_papers",
            description=(
                "Provides information about RAG"
            ),
        ),
    ),
]

In [32]:
langchain_tools = [t.to_langchain_tool() for t in query_engine_tools]

In [38]:
system_context = "You are a researcher in Deep Learning.\
You will answer questions about transformers, cnn, RAG"

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0, streaming=True)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_context,
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(llm, langchain_tools, prompt)

In [34]:
agent_executor = AgentExecutor(agent=agent, tools=langchain_tools, verbose=True, return_intermediate_steps=True, handle_parsing_errors=True, max_iterations=5)

In [36]:
question =  "What is RAG?"

response = agent_executor.invoke({"input": question})

print("\n Response:", response['output'])



> Entering new AgentExecutor chain...

Invoking: `RAG_papers` with `{'input': 'What is RAG?'}`


RAG stands for Recognition Graph.RAG, in the context of deep learning, typically refers to "Retrieval-Augmented Generation," which is a method for combining the dense vector retrieval of documents (or other data) with a sequence-to-sequence model for natural language generation tasks. However, the response I received indicates "Recognition Graph," which seems to be an unrelated concept or a misunderstanding.

If you are indeed asking about Retrieval-Augmented Generation, it is a framework that enhances the capabilities of language models by allowing them to access external knowledge sources. This is particularly useful for tasks where the model needs to generate responses based on information that is not contained within its parameters but can be found in reference documents or databases.

Would you like to know more about Retrieval-Augmented Generation (RAG) in the context of deep learni